In [ ]:
#CHDIR COMMAND ONLY FOR JUPYTER
import os
import sys
sys.path.append("/home/srichers/software/Emu_scripts/data_reduction")
import yt
import numpy as np
import matplotlib.pyplot as plt
import h5py
import amrex_plot_tools as amrex
import emu_yt_module as emu
import spin_flip_tools as sft
import glob
import concurrent

# For ease of development, reload the relevant modules to make sure they are up to date.
import importlib
importlib.reload(sft)

In [ ]:
data_base_directory = "/home/srichers/data/1D_fiducial/"
directory_list = sorted(glob.glob(data_base_directory+"plt*"))

output_filename = data_base_directory + "spin_flip.h5"

basis_theta = 0
basis_phi = 0

In [ ]:
if os.path.exists(output_filename):
    os.remove(output_filename)

for d in directory_list:
    print(d)
    with concurrent.futures.ProcessPoolExecutor(max_workers=1) as executor:
        executor.submit(sft.interact, d, output_filename, basis_theta, basis_phi)

In [ ]:
# open the hdf5 file with read permissions
data = h5py.File(output_filename,"r")

# show the available datasets
# note - use h5ls in the terminal to peek inside a hdf5 file
list(data.keys())

In [ ]:
# Plot H_LR at t=0
z = np.array(data["z(cm)"])
H_LR = np.array(data["H_LR(eV)"])
plt.plot(z, H_LR[0])

In [ ]:
# plot the average across time
t = np.array(data["t(s)"])
H_LR_avg = np.average(H_LR, axis=1)
plt.plot(t, H_LR_avg)